Nous allons maintenant faire une prédiction et dire si une phrase est affirmative ou négative ! 
pour cela on utilise une dataset de puis imdb ! 
nous allons commencer par lire dataset   


In [2]:
import tensorflow as tf 
import matplotlib.pyplot as plt
import numpy as np 
import time 
from scipy.spatial.distance import cdist
#from tensorflow.keras.datasets import imdb
import imdb 

#  pour le modéle  ! 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences




SyntaxError: EOL while scanning string literal (imdb.py, line 42)

In [3]:
x_train_text, y_train = imdb.load_data(train=True)
x_test_text, y_test = imdb.load_data(train=False)

print("Train-set size: ", len(x_train_text))
print("Test-set size:  ", len(x_test_text))

data_text = x_train_text + x_test_text

 # nous allons maintenant  transformer notre data en tokens ! 
num_words=10000
tokenizer=Tokenizer(num_words=num_words)

tokenizer.fit_on_texts(data_text)

# maintenat nous devons utiliser le tokenizers pour rendre nis x_train_text en x_train_tokens
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)#texts_to_sequences fait l'affaire! 
x_test_tokens= tokenizer.texts_to_sequences(x_test_text)


#on vient de normaliser nos données maintenant nous devons les rendre equilibrés ! de sorte que les input aurons la meme taille *
#pour cela nous devons maximiser le nombre de tokens ! 
#calculons le max 
#ici nous prenons tous les tokens ! (test+train) et ceci pour chaque longueur de token ! et le rendre un array puis voir 
#la moyenne de celui si avec np.main(num_tokens)
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
np.mean(num_tokens)

np.max(num_tokens)
#et donx le maximum de tokens (mots) dans notre cas est 

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)#moyenne+2*l'ecart type ! 
max_tokens = int(max_tokens)#partie entriére 
max_tokens #la taille max  de la sequence est donc : 545 
tokenizer.word_index



Train-set size:  0
Test-set size:   0


C:\Users\TRIUMPHAL\AppData\Local\conda\conda\envs\TensorProject\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\TRIUMPHAL\AppData\Local\conda\conda\envs\TensorProject\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: zero-size array to reduction operation maximum which has no identity

maintenant en ce lance au padding et trancking ! alors aprés avoir peut etre alarger notre token  ! il faut remplir les vides avec des 0 mais est ce que au debut ! ou bien à la fin  ! si pad=pred (au debut) sinon si pad=post alors à la fin! 
on choisit avant ! 
pad=pre

In [ ]:
pad="pre"
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad) #et voila du x_train_text => x_train_tokens=>x_train_pad 
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

x_train_pad.shape
x_test_pad.shape
#pour bien comprendre voila un affichage : 
x_train_pad[1]

In [ ]:
#conversion  token-integer =>words
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text



In [ ]:
x_train_text[3]


In [ ]:
#creation the model  !

model=Sequential()
embedding_size=8
couche_1 = Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                     name='layer_embedding')
#cette couche est obligatoirement la premiére du model ! #je pense que c est equivalant a les scaller ! 
#Turns positive integers (indexes) into dense vectors of fixed size. eg. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]
model.add(couche_1)

couche_2=LSTM(32,activation='tanh',return_sequences=True)
model.add(couche_2)

couche_3=LSTM(16,activation='tanh',return_sequences=True)
model.add(couche_3)

couche_3=LSTM(8,activation='tanh')
model.add(couche_3)

couche_4=Dense(1, activation='sigmoid')
model.add(couche_4)

#compile : 
optimizer=tf.keras.optimizers.Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


model.summary()


In [ ]:
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=64) #training
result = model.evaluate(x_test_pad, y_test)#testing 

print("Accuracy: {0:.2%}".format(result[1]))



In [ ]:
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Not a good movie!"
text8 = "This movie really sucks! Can I get my money back please?"
texts = [text1, text2, text3, text4, text5, text6, text7, text8]
tokens = tokenizer.texts_to_sequences(texts)

tokens_pad = pad_sequences(tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

tokens_pad.shape

In [ ]:
model.predict(tokens_pad)

